In [3]:
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from mcl.mcl_clustering import networkx_mcl

import networkx as nx

In [4]:
def displayGraph(graph, title='', color_values = None):
    pos = nx.spring_layout(graph)
    nx.draw_networkx_edges(graph, pos, arrows=True)
    nx.draw_networkx_nodes(graph, pos, cmap=plt.cm.Blues, node_color = color_values, alpha=0.8)

    nx.draw_networkx_labels(graph,pos)
    plt.title(title)
    plt.show()

In [11]:
# try loading file
sanfrancisco = pickle.load(open('.\Neo4j Data\San_Francisco_Physicians.p', 'rb'))


NameError: name 'sanfracisco' is not defined

In [15]:
print sanfrancisco[0]

[<Neo4j Node: http://ec2-52-23-203-124.compute-1.amazonaws.com:7474/db/data/node/5469>, <Neo4j Relationship: http://ec2-52-23-203-124.compute-1.amazonaws.com:7474/db/data/relationship/38995045>, <Neo4j Node: http://ec2-52-23-203-124.compute-1.amazonaws.com:7474/db/data/node/1176839>]


In [17]:
from utils import parse_neo4j_to_nx
        
graph = parse_neo4j_to_nx(sanfrancisco)
displayGraph(graph)

KeyboardInterrupt: 

In [ ]:
M, clusters = networkx_mcl(graph, expand_factor=1)
print clusters